In [2]:
import pymysql
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from qkids.DatabasePool import get_schedule_connection, get_course_connection,get_teacher_connection

In [26]:
conn = get_schedule_connection()
def get_free_teacher_capacity_dataframe(slot):
    rooms = get_free_teachers(slot)
    teachers = tuple(data.teacher_id)
    return get_chapter(teachers), get_room_type(teachers)
   
def get_free_teachers(slot):
#     sql  = "select r.id, schedule_id, room_type_id, course_id, lesson_id, student_count, teacher_id \
#     from schedules s join rooms r on s.id = r.schedule_id and r.status = 0 and klass_id is null \
#     and r.is_internal = 0 and ((teacher_id > 0 and student_count =0) or (teacher_id = 0 and student_count > 0)) \
#      where begin_at = %r "  % slot
    sql  = "select r.id, schedule_id, room_type_id, course_id, lesson_id, student_count, teacher_id \
        from schedules s join rooms r on s.id = r.schedule_id and r.status = 0 and klass_id is null \
        and r.is_internal = 0 and teacher_id > 0 and student_count = 0 \
         where begin_at = %r "  % slot
    return pd.read_sql(sql, get_schedule_connection(), index_col='id')

def get_chapter(teachers):
    sql = f"select teacher_id, chapter_id, status from teacher_chapter_tables where teacher_id in {str(teachers)} and status > 0"
    df = pd.read_sql(sql, get_teacher_connection())
    df = df.pivot_table(values='status', index = 'teacher_id', columns='chapter_id', fill_value=0)
    return df

def get_room_type(teachers):
    sql = f"select teacher_id, room_type, status from teacher_room_types where teacher_id in {str(teachers)} and status > 0"
    df = pd.read_sql(sql, get_teacher_connection())
    df = df.pivot_table(values='status', index = 'teacher_id', columns='room_type', fill_value=0)
    return df

### 为什么会删除房间

In [27]:
room_id = 7122459
sql = "select begin_at, schedule_id, room_type_id, lesson_id from schedules s join rooms r on s.id = r.schedule_id \
    where r.id = 7122459"
with conn.cursor() as cur:
    cur.execute(sql)
    room = cur.fetchone()
print("ROOM INFO:",room)
slot = room[0].strftime('%Y-%m-%d %H:%M:00')
print(slot)

df_chapter, df_room_type = get_free_teacher_capacity_dataframe(slot)
#
course_conn = get_course_connection()
sql = "select id, chapter_id from lessons where published = 1 and deleted_at is null"
lessons = pd.read_sql(sql, course_conn, index_col='id')
chapter_id = lessons.loc[room[3], 'chapter_id']

## 可用老师为
print('可用老师为')
t1 = df_chapter.loc[df_chapter.loc[:, chapter_id] > 0].index
t2 = df_room_type.loc[df_room_type.loc[:, room[2]] > 0].index
print(t1.intersection(t2))

ROOM INFO: (datetime.datetime(2019, 6, 3, 19, 20), 2964373, 2, 1674)
2019-06-03 19:20:00
可用老师为
Int64Index([  4293,   4630,   6316, 100190, 100644, 100840, 100947, 100953,
            101859, 101902, 102405, 102488, 102674, 103213, 103430, 103504,
            105686, 106956, 110949, 111052, 111873, 112139, 112148, 112523,
            112818, 112873, 117047, 119725, 128249],
           dtype='int64', name='teacher_id')


### 为什么这个L 没有课了

### 空闲的老师

In [5]:
sql  = "select r.id, schedule_id, room_type_id, course_id, lesson_id, student_count, teacher_id \
from schedules s join rooms r on s.id = r.schedule_id and r.status = 0 and klass_id is null \
and r.is_internal = 0 and ((teacher_id > 0 and student_count =0) or (teacher_id = 0 and student_count > 0)) \
 where begin_at = %r "  % slot
data = pd.read_sql(sql, conn)
rooms = data.loc[data['teacher_id'] == 0]

### 获取老师的chapter 和  room type

In [6]:
teacher_conn = get_teacher_connection()
tlist = str(tuple([i for i in data['teacher_id'].values if i > 0]))
sql = "select teacher_id, chapter_id, status from teacher_chapter_tables where teacher_id in {} and status > 0".format(tlist)
with teacher_conn.cursor() as cur:
    cur.execute(sql)
    chapter_records = cur.fetchall()
chapters = pd.DataFrame(columns = ('a',), dtype='int8')
for row in chapter_records:
    chapters.loc[row[0], row[1]] = int(row[2])
chapters.drop('a', axis=1,inplace=True)
    
sql = "select teacher_id, room_type, status from teacher_room_types where teacher_id in {} and status > 0".format(tlist)
with teacher_conn.cursor() as cur:
    cur.execute(sql)
    room_type_records = cur.fetchall()
roomtypes = pd.DataFrame(columns = ('a',), dtype='int8')    
for row in room_type_records:
    roomtypes.loc[row[0],row[1]] = int(row[2])
roomtypes.drop('a', axis = 1, inplace=True)

#### Lesson 数据  转chapter

In [7]:
course_conn = get_course_connection()
sql = "select id, chapter_id from lessons where published = 1 and deleted_at is null"
lessons = pd.read_sql(sql, course_conn, index_col='id')

###  房间匹配

In [8]:
def match(room_chapter, room_type):
    rt = set(roomtypes.loc[roomtypes[room_type] == 2].index.tolist())
    c = set(chapters.loc[chapters[room_chapter] == 2].index.tolist())
    print(rt&c)
    rt = set(roomtypes.loc[roomtypes[room_type] == 2].index.tolist())
    c = set(chapters.loc[chapters[room_chapter] == 1].index.tolist())
    print(rt&c)
    rt = set(roomtypes.loc[roomtypes[room_type] == 1].index.tolist())
    c = set(chapters.loc[chapters[room_chapter] == 2].index.tolist())
    print(rt&c)
    rt = set(roomtypes.loc[roomtypes[room_type] == 1].index.tolist())
    c = set(chapters.loc[chapters[room_chapter] == 1].index.tolist())
    print(rt&c)

In [9]:
for index, room in rooms.iterrows():
    chapter = lessons['chapter_id'][room['lesson_id']]
    print('-' *32)
    print("room[{}]:".format(room['id']))
    match(chapter, room['room_type_id'])

--------------------------------
room[7031682]:
set()
{4610, 104963, 102405, 526, 101395, 3604, 102419, 4630, 5654, 101912, 106521, 111643, 101405, 200224, 105508, 113188, 128036, 120871, 150052, 1066, 107563, 112683, 101933, 122926, 105524, 100919, 110135, 115767, 174136, 166972, 110653, 103998, 109631, 1089, 6211, 100420, 105541, 112195, 134211, 162377, 101965, 3155, 105046, 165979, 136285, 138846, 171618, 105575, 103016, 107111, 112235, 110189, 4718, 106606, 112750, 128621, 3191, 155256, 163450, 4219, 123003, 5755, 6782, 175233, 131722, 4747, 3213, 112269, 110740, 109205, 113301, 4759, 129177, 1690, 6810, 169113, 104603, 106153, 170666, 105645, 112814, 124077, 130734, 113329, 3764, 103093, 105141, 126646, 127669, 136372, 108730, 115898, 238262, 105663, 711, 116935, 108235, 135375, 125137, 107221, 1238, 1239, 111829, 100061, 110815, 132833, 141537, 106214, 109286, 170729, 127722, 2284, 103660, 102126, 121075, 104182, 109304, 168185, 109313, 128770, 112899, 205573, 119049, 107276, 111

### 需要扩充的schedule

In [10]:
sql  = "select schedule_id, room_type_id, course_id, lesson_id \
from schedules s join rooms r on s.id = r.schedule_id and r.status = 0 and klass_id is null \
and r.is_internal = 0 where begin_at = %r group by schedule_id having sum(max_student_count) = \
sum(student_count)"  % slot
df = pd.read_sql(sql, conn)

In [11]:
for index, row in df.iterrows():
    chapter = lessons['chapter_id'][row['lesson_id']]
    print('-' *32)
    print("Schedule[{}]:".format(row['schedule_id']))
    match(chapter, row['room_type_id'])  

--------------------------------
Schedule[2907662]:
{111873, 110949, 112873, 112139, 111052, 112523, 112818, 112148, 117047, 128249}
set()
set()
set()
